In [1]:
# Import cfbd library, pandas, and establish API connection to Games and Lines instance

import cfbd
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'XXXXXXX'
configuration.api_key_prefix['Authorization'] = 'Bearer'
api_config = cfbd.ApiClient(configuration)

api_games_instance = cfbd.GamesApi(api_config)
api_lines_instance = cfbd.BettingApi(api_config)
api_rankings_instance = cfbd.RankingsApi(api_config)

In [2]:
## Pull all games in 2024 played/unplayed

games = []

games_response = api_games_instance.get_games(year=2024, season_type = 'regular')
games = [*games, *games_response]

In [3]:
# Flatten games API response and filter only to needed fields

games = [
    dict(
        game_id=g.id,
        season=g.season,
        week=g.week,
        start_date=g.start_date,
        start_time_tbd=g.start_time_tbd,
        neutral_site = g.neutral_site,
        home_id = g.home_id,
        home_team=g.home_team,
        home_conference=g.home_conference,
        home_points=g.home_points,
        home_pregame_elo=g.home_pregame_elo,
        away_id=g.away_id,
        away_team=g.away_team,
        away_conference=g.away_conference,
        away_points=g.away_points,
        away_pregame_elo=g.away_pregame_elo
    ) for g in games]

In [4]:
# Create fields in order to calculate running totals of wins/losses by week

games_2024_df = pd.DataFrame.from_records(games)
games_2024_df.to_csv('games_2024.csv')

games_2024_df.loc[games_2024_df['home_points'] > games_2024_df['away_points'], 'home_win2'] = 1
games_2024_df.loc[games_2024_df['away_points'] > games_2024_df['home_points'], 'home_win2'] = 0
games_2024_df.loc[games_2024_df['home_points'] < games_2024_df['away_points'], 'home_loss2'] = 1
games_2024_df.loc[games_2024_df['away_points'] < games_2024_df['home_points'], 'home_loss2'] = 0

games_2024_df.loc[games_2024_df['home_points'] < games_2024_df['away_points'], 'away_win2'] = 1
games_2024_df.loc[games_2024_df['away_points'] < games_2024_df['home_points'], 'away_win2'] = 0
games_2024_df.loc[games_2024_df['home_points'] > games_2024_df['away_points'], 'away_loss2'] = 1
games_2024_df.loc[games_2024_df['away_points'] > games_2024_df['home_points'], 'away_loss2'] = 0

games_2024_df.loc[games_2024_df['home_points'].isnull(), 'game_played_home'] = 0
games_2024_df.loc[games_2024_df['home_points'].notnull(), 'game_played_home'] = 1

games_2024_df.loc[games_2024_df['away_points'].isnull(), 'game_played_away'] = 0
games_2024_df.loc[games_2024_df['away_points'].notnull(), 'game_played_away'] = 1

games_2024_df = games_2024_df.drop(games_2024_df[['game_id','start_date','start_time_tbd','neutral_site','home_id','home_conference','home_points','home_pregame_elo',
                                                  'away_id','away_conference','away_points','away_pregame_elo']],axis=1)

In [5]:
# Calculate running wins/losses for each team

records_df_a = games_2024_df[['season','week','game_played_home','home_team','home_win2','home_loss2']]
records_df_b = games_2024_df[['season','week','game_played_away','away_team','away_win2','away_loss2']]

records_df_a = records_df_a.sort_values(['season', 'week','home_team'])
records_df_b = records_df_b.sort_values(['season', 'week','away_team'])

records_df_a = records_df_a.rename(columns={"home_team": "team","game_played_home": "game_played"})
records_df_b = records_df_b.rename(columns={"away_team": "team","game_played_away": "game_played"})


records_df = pd.concat([records_df_a,records_df_b])
records_df = records_df.sort_values(['season', 'week','team'])
records_df.fillna(0,inplace=True)

records_df['running_wins'] = records_df['home_win2'] + records_df['away_win2']
records_df['running_losses'] = records_df['home_loss2'] + records_df['away_loss2']

records_df['win_total'] = records_df.groupby(['team'])['running_wins'].cumsum()
records_df['loss_total'] = records_df.groupby(['team'])['running_losses'].cumsum()

In [6]:
# Account for bye weeks and completed games for the current week

records_df['week_lag'] = records_df.groupby(['team'])['week'].shift(-1) - records_df['week']

## BYE WEEK
records_df_duplicated_1 = records_df[records_df['week_lag'] == 2].copy()
records_df_duplicated_1['week'] = records_df_duplicated_1['week']+1

## COMPLETED GAME CURRENT WEEK
records_df_duplicated_2 = records_df[records_df['week_lag'].isnull()].copy()
records_df_duplicated_2['week'] = records_df_duplicated_2['week']+1

records_df_final = pd.concat([records_df,records_df_duplicated_1,records_df_duplicated_2])

In [7]:
# Create CSV with records for every week for every team

records_df_final.to_csv('records_2024.csv',index=False)

In [8]:
# Make API call for rankings

api_rankings_instance = cfbd.RankingsApi(api_config)

rankings_raw = []

for week in range(1,17):
    rankings_response = api_rankings_instance.get_rankings(year=2024, week=week)
    rankings_raw = [*rankings_raw, *rankings_response]

In [9]:
# Flatten rankings data and filter to only AP top 25
# Create CSV with AP rankings for every week

ranks=[]

for ranking in rankings_raw:
    for p in ranking.polls:
        for r in p.ranks:
            ranks.append(dict(
                year=ranking.season,
                week=ranking.week,
                poll=p.poll,
                rank=r.rank,
                school=r.school
            ))
            
rankings_df = pd.DataFrame.from_records(ranks)
rankings_AP_df = rankings_df.loc[rankings_df['poll']=='AP Top 25']
rankings_AP_df.to_csv('rankings_2024.csv',index=False)

In [10]:
# Pull lines for all 2024 games played/unplayed

lines=[]

lines_response = api_lines_instance.get_lines(year=2024, season_type = 'regular')
lines = [*lines, *lines_response]

In [11]:
# Join betting lines data with games data, only rows with both a spread and over/under line

# Current betting lines provider: Bovada

for game in games:
    game_lines = [l for l in lines if l.id == game['game_id']]

    if len(game_lines) > 0:
        game_line = [l for l in game_lines[0].lines if l.provider == 'Bovada']

        if len(game_line) > 0 and game_line[0].spread is not None:
            game['spread'] = float(game_line[0].spread)
    
        over_under = [l for l in game_lines[0].lines if l.provider == 'Bovada']
        
        if len(over_under) > 0 and game_line[0].over_under is not None:
            game['over_under'] = float(game_line[0].over_under)

In [12]:
# Convert games and lines merged data into pandas dataframe
# Drop null columns for anything but the score
# Filter out any weeks unplayed except for upcoming week

import numpy as np

games_lines_df = pd.DataFrame.from_records(games)

games_lines_df.dropna(subset=['home_pregame_elo','away_pregame_elo'],inplace=True)

max_week = games_lines_df.loc[games_lines_df['home_points'].notnull()]
max_week = max_week['week'].max()

max_week_check = games_lines_df.loc[games_lines_df['week']==max_week,['home_points']]

if(max_week_check.isna().any().any()):
    games_lines_df = games_lines_df.loc[games_lines_df['week']<=max_week]
else:
    games_lines_df = games_lines_df.loc[games_lines_df['week']<=max_week+1]

In [13]:
# Merge with team info dataframe

team_info_df = pd.read_csv('team_info.csv')

games_lines_df = games_lines_df.merge(team_info_df,
                                      left_on = ['home_id'],
                                      right_on = ['id']).merge(team_info_df,
                                                               left_on = ['away_id'],
                                                               right_on = ['id'],
                                                               suffixes = ['_home','_away'])
                                                                                                                        

games_lines_df = games_lines_df.drop(games_lines_df[['id_home','id_away','conference_home','conference_away']],axis=1)

In [14]:
# Create CSV for 2024 games with lines data

games_lines_df.to_csv('games_with_lines_2024.csv',index=False)